#  Generating GRCh38 Medical Genes Benchmark
This document details the steps to generate the medical relevant genes benchmark. The input files are also in this gitlab repo and all paths are from the repo top level directory.

1) Union of Mandelker et al Supplementary Table 13, COSMIC Cancer Gene Census, Steve Lincoln Medical Gene Lists -- Mandelker_COSMIC_Lincoln_gene_symbols.tsv

2) Look up coordinates for gene symbols in ENSEMBLE GRCh38 Human Genes v100 -- lookup_MRG_symbol_coordinates_ENSEMBL.R

3) Find overlap of genes with HG002 v4.2.1, then add slop and find overlap with HG002 hifiasm v0.11 dip.bed

4) Find genes that were < 90% overlap with GRCh38 v4.2.1 and fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh38 dip.bed, find union of GRCh37 and GRCh38 MRG lists, then add genes that are unique to GRCh37 but still fully fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh38 dip.bed -- find_coordinates_of_MRG_GRCh37_GRCh38_union.R

5) Use coordinates for benchmark then remove 
    - homopolymers and imperfect homopolymers > 20
    - SVs with 50bp flanking and overlapping tandem repeats
    - hifiasm error
    - GRCh38 GAPs
    - Remove partially covered tandem repeats
    - Remove MHC region

## Find overlap with v4.2.1 in ENSEMBL gene annotations

In [ ]:
python find_overlap_per_gene.py --input_benchmark HG002_GRCh38_1_22_v4.2.1_benchmark_noinconsistent.bed --input_genes GRCh38_ENSEMBL_100_Union_MRG_coordinates.bed --output GRCh38_ENSEMBL_100_Union_MRG_coordinates_overlap_with_v4.2.1_benchmark.bed

## Create flanking sequence bed for MRG candidates

In [ ]:
bedtools slop -i GRCh38_ENSEMBL_100_Union_MRG_coordinates.bed -g human.hg38.genome -b 20000 > GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp.bed

## Find breaks in dip.bed in MRG candidates

In [ ]:
python find_overlap_per_gene.py --input_benchmark HG002v11-align2-GRCh38.dip_check_for_breaks.bed --input_genes GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp.bed --output  GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp_check_for_breaks_in_dip.bed

## Find coordinates of ENSEMBL gene annotations with flanking sequence and overlapping segdups

In [ ]:
python find_flanking_and_segdups_coverage_GRCh38.py --input_genes GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp.bed --output GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp_flanking_and_segdups_coordinates.bed

## Append gene names 

Add column from GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_slop20000bp.bed to GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_slop20000bp_flanking_and_segdups_coordinates.bed to create GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names.bed

In [ ]:
cat HG002v11-align2-GRCh38.dip.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG002v11-align2-GRCh38.dip_sorted.bed

## Find overlap of HG002 GRCh38 hifiasm v0.11 of ENSEMBL gene annotations with flanking sequence and overlapping segdups

In [ ]:
python find_overlap_per_gene_w_segdups_coordinates.py  --input_benchmark HG002v11-align2-GRCh38.dip_sorted.bed --input_genes GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names.bed --output GRCh38_ENSEMBL_100_Union_MRG_coordinates_slop20000bp_flanking_and_segdups_coordinates_w_gene_names_overlap_HG002v11-align2-GRCh38.bed

## Create HG002_GRCh37_overlap_v4.2.1_and_hifiasm.tsv 

Combine all columns of GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_overlap_with_v4.2.1_benchmark.bed, appending columns 5 and 6 of GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_slop20000bp_overlapping_segdups_gene_names_overlap_HG002v11-align2-GRCh38.bed, then append column 4 of GRCh38_ENSEMBL_100_Mandelker_COSMIC_Lincoln_coordinates_slop20000bp_check_for_breaks_in_dip.bed

GRCh38_overlap_v4.2.1_and_hifiasm.tsv column names are chrom, start, end, gene, bp_overlap_v4.2.1, percent_overlap_v4.2.1, bp_flanking_plus_segdups_overlap_hifiasm, percent_flanking_plus_segdups_overlap_hifiasm, flanking_breaks_in_dip_bed

## HG002_GRCh37_MRG.bed is from find_coordinates_of_MRG_GRCh37_GRCh38_union.R

## Then run bedtools merge

In [ ]:
bedtools subtract -a HG002_GRCh38_MRG.bed -b GRCh38_MHC.bed.gz > HG002_GRCh38_MRG_no_MHC.bed 

bedtools merge -i HG002_GRCh38_MRG_no_MHC.bed > HG002_GRCh38_MRG_merged.bed

## Remove homopolymers > 20bp

In [ ]:
bedtools subtract -a HG002_GRCh38_MRG_merged.bed -b Regions_to_exclude_from_small_variant_benchmark/GRCh38_SimpleRepeat_homopolymer_gt20_slop5.bed.gz > HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_SimpleRepeat_homopolymer_gt20_slop5.bed

## Remove imperfect homopolymers > 20bp

In [ ]:
bedtools subtract -a HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_SimpleRepeat_homopolymer_gt20_slop5.bed -b Regions_to_exclude_from_small_variant_benchmark/GRCh38_SimpleRepeat_imperfecthomopolgt20_slop5.bed.gz > HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_SimpleRepeat_imperfecthomopolgt20_slop5.bed

## SVs with 50bp flanking and overlapping tandem repeats

HG002v11-align2-GRCh38.dip.vcf.gz is from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.01.00/GRCh38/hifiasm_v0.11/HG002v11-align2-GRCh38.dip.vcf.gz

GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz is from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh38/LowComplexity/GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz

In [ ]:
gunzip -c HG002v11-align2-GRCh38.dip.vcf.gz | awk 'length($4)>49 || length($5)>49' | awk '{FS=OFS="\t"} {print $1,$2-1,$2+length($4)}' > HG002v11-align2-GRCh38.dip_SVsgt49bp.bed

intersectBed -wa -a GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b HG002v11-align2-GRCh38.dip_SVsgt49bp.bed | multiIntersectBed -i stdin HG002v11-align2-GRCh38.dip_SVsgt49bp.bed |  awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' | mergeBed -i stdin -d 1000 > HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

bedtools subtract -a HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_SimpleRepeat_imperfecthomopolgt20_slop5.bed -b Regions_to_exclude_from_small_variant_benchmark/HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed > HG002_GRCh38_Union_MRG_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

## Remove hifiasm error on chr21

In [ ]:
bedtools subtract -a HG002_GRCh38_Union_MRG_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed -b Regions_to_exclude_from_small_variant_benchmark/GRCh38_hifiasm_error.bed > HG002_GRCh38_Union_MRG_temp_subtract_hifiasm_error.bed

## Remove GRCh38 GAPs

In [ ]:
bedtools subtract -a HG002_GRCh38_Union_MRG_temp_subtract_hifiasm_error.bed -b Regions_to_exclude_from_small_variant_benchmark/GRCh38_MRG_GAPs.bed > HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_MRG_GAPs.bed

## Sort

In [ ]:
cat HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_MRG_GAPs.bed | sed 's/^chr//' | sort -k1,1n -k2,2n | sed 's/^/chr/' > HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_MRG_GAPs_sorted.bed

## Remove partially covered tandem repeats


In [ ]:
complementBed -i HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_MRG_GAPs_sorted.bed -g human.b38.genome | intersectBed -wa -a GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b stdin | subtractBed -a HG002_GRCh38_Union_MRG_temp_subtract_GRCh38_MRG_GAPs_sorted.bed -b stdin > HG002_GRCh38_Union_MRG_temp_subtract_partial_tandem_repeats.bed

## Remove MHC as it is covered in MHC benchmark

In [ ]:
bedtools subtract -a HG002_GRCh38_Union_MRG_temp_subtract_partial_tandem_repeats.bed -b GRCh38_MHC.bed.gz > HG002_GRCh38_difficult_medical_gene_smallvar_benchmark.bed

cat HG002_GRCh38_difficult_medical_gene_smallvar_benchmark.bed | awk '{sum+=$3-$2} END {print sum}'

# HG002v11-align2-GRCh38.dip.vcf.gz is retrieved from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.01.00/GRCh38/hifiasm_v0.11/HG002v11-align2-GRCh38.dip.vcf.gz
bedtools intersect -a HG002v11-align2-GRCh38.dip.vcf.gz -b HG002_GRCh38_MRG_merged.bed -header > HG002_GRCh38_difficult_medical_gene_smallvar_benchmark.vcf

bgzip HG002_GRCh38_difficult_medical_gene_smallvar_benchmark.vcf

tabix HG002_GRCh38_difficult_medical_gene_smallvar_benchmark.vcf.gz

## SV benchmark

In [ ]:
# Find SVs MRG benchmark gene coordinates
bedtools intersect -a HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed -b HG002_GRCh38_MRG_merged.bed > HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed

# Subset to SVs only gt49bp 
cat HG002v11-align2-GRCh38.dip.vcf | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>49 || length($5)-length($4)>49 || length($6)-length($4)>49) print}' | intersectBed -a HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed -b stdin -c | awk '$4>0' | cut -f1-3 > HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed 

# Find isolated SVs  

cat HG002v11-align2-GRCh38.dip.vcf | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' | intersectBed -a HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed -b stdin -c | awk '$4==1' | cut -f1-3 > HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_isolated.bed 

# Find complex SVs  

cat HG002v11-align2-GRCh38.dip.vcf | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' | intersectBed -a HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed -b stdin -c | awk '$4>1' | cut -f1-3 > HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed 

# Remove complex SVs from MRG gene candidate coordinates and remove GAPs 
bedtools subtract -a HG002_GRCh38_MRG_merged.bed -b HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed | bedtools subtract -a stdin -b  Regions_to_exclude_from_small_variant_benchmark/GRCh38_MRG_GAPs.bed > HG002_GRCh38_MRG_draft_SV_benchmark_temp.bed

#HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed  from the SV benchmark bed:
# Find tandem repeats and homopolymers that have multiple indels >9bp, since these can add up to >49bp and should be subtracted from the benchmark SV bed
cat HG002v11-align2-GRCh38.dip.vcf | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' | intersectBed -a GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b stdin -c | awk '$4>1' | cut -f1-3 | intersectBed -v -a stdin -b HG002v11-align2-GRCh38.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed > HG002v11-align2-GRCh38.dip_complexindelsgt9bpinRepeats.bed

bedtools subtract -a HG002_GRCh38_MRG_draft_SV_benchmark_temp.bed -b HG002v11-align2-GRCh38.dip_complexindelsgt9bpinRepeats.bed > HG002_GRCh38_MRG_draft_SV_benchmark.bed

cat HG002_GRCh38_MRG_draft_SV_benchmark.bed | awk '{sum+=$3-$2} END {print sum}'

# Decompose for truvari comparison
#vt decompose -s HG002v11-align2-GRCh37.dip.vcf -o HG002v11-align2-GRCh37.dip_decomposed.vcf
#python script to remove ambiguous (non-ACTGN) REF

## remove ambiguous (non-ACGTN) in REF field. Adjust path to where you keep this file
python fix_reference_allele_py.py --input_vcf_file HG002v11-align2-GRCh38.dip.vcf --output_file HG002v11-align2-GRCh38._noambig.dip.vcf

## zip for bcftools
bgzip -c HG002v11-align2-GRCh38._noambig.dip.vcf > HG002v11-align2-GRCh38._noambig.dip.vcf.gz

tabix HG002v11-align2-GRCh38._noambig.dip.vcf.gz

## split multiallelic to biallelic
#bcftools norm -m- (multiallelic split)

bcftools norm -m- -Oz HG002v11-align2-GRCh38._noambig.dip.vcf.gz -o HG002v11-align2-GRCh38._noambig_norm_m.vcf.gz

## left align and normalize indels. Adjust to path where your reference.fa is located. 
#bcftools norm -f  (normalization)

bcftools norm -f /Users/jmw7/v4.1_development/GRCh38/Reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fa -Oz -o HG002v11-align2-GRCh38._noambig_norm_mf.vcf.gz HG002v11-align2-GRCh38._noambig_norm_m.vcf.gz


## remove duplicate records
#bcftools norm -d  (remove duplicate records)

bcftools norm -d none -Oz HG002v11-align2-GRCh38._noambig_norm_mf.vcf.gz -o HG002v11-align2-GRCh38._noambig_norm_mfd.vcf.gz

## remove MHC region. Adjust to path where MHC.bed is located. 

bedtools subtract -header -a HG002v11-align2-GRCh38._noambig_norm_mfd.vcf.gz -b /Users/jmw7/genome-stratifications-v2.0/GRCh38/OtherDifficult/GRCh38_MHC.bed.gz | bgzip -c >  HG002v11-align2-GRCh38._noambig_norm_mfd_noMHC.vcf.gz

## intersect w/ benchmark bed and subset to >39bp in REF or ALT fields. Adjust to path of benchmark.bed
#intersect w/ MRG target regions and subset >39 bp

bedtools intersect -header -a HG002v11-align2-GRCh38._noambig_norm_mfd_noMHC.vcf.gz -b HG002_GRCh38_MRG_draft_SV_benchmark.bed | awk '$1 ~ /^#/ || length($4)>39 || length($5)>39' | bgzip -c > HG002v11-align2-GRCh38._noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz

## index vcf, required by truvari
tabix -p vcf HG002v11-align2-GRCh38._noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz
